# Code for running the regression model for the effect of key moments on the flow of momentum

In [ ]:
#import the PC and PCV regression dataset
import pickle
infile = open('input PC regression key moments on Momentum.p','rb')
df_keys = pickle.load(infile)
infile.close()

In [ ]:
#add the score difference for the home and away team
matches = df_keys.match_id.unique()
home_goals_complete = []
away_goals_complete = []
for i in range(len(matches)):
    df_shots = df_keys.loc[df_keys['match_id'] == matches[i]]    
    goals = []
    for j in range(len(df_shots)):
        if df_shots.iloc[j]['type_id'] == '16':
            goals.append([int(df_shots.iloc[j]['minute']) + int(df_shots.iloc[j]['seconds'])/60, df_shots.iloc[j]['team']])
    home_goals = []
    away_goals = []
    for j in range(len(df_shots)):
        home = 0
        away = 0
        for x in range(len(goals)):
            if (int(df_shots.iloc[j]['minute']) + int(df_shots.iloc[j]['seconds'])/60) >= goals[x][0] and goals[x][1] == 'Home':
                home = home + 1
            elif (int(df_shots.iloc[j]['minute']) + int(df_shots.iloc[j]['seconds'])/60) >= goals[x][0] and goals[x][1] == 'Away':
                away = away + 1
        home_goals.append(home)
        away_goals.append(away)
    home_goals_complete.extend(home_goals)
    away_goals_complete.extend(away_goals)

df_keys['home score'] = home_goals_complete
df_keys['away score'] = away_goals_complete
df_keys['score difference home'] = df_keys['home score'] - df_keys['away score']
df_keys['score difference away'] = df_keys['away score'] - df_keys['home score']

                
                

In [ ]:
#add the time left
time_left = []
for i in range(len(df_keys)):
    time = int(df_keys.iloc[i]['minute']) + int(df_keys.iloc[i]['seconds'])/60
    time_left.append(90-time)
df_keys['time left'] = time_left

In [ ]:
#add the amount of time elapsed since the previous shot or goal
matches = df_keys.match_id.unique()
last_moment = []
for i in range(len(matches)):
    df_match = df_keys.loc[df_keys['match_id'] == matches[i]]
    moments = []
    for j in range(len(df_match)):
        moments.append(int(df_match.iloc[j]['minute']))
    for j in range(len(df_match)):
        moments_now = [x for x in moments if x<int(df_match.iloc[j]['minute'])]
        if len(moments_now) == 0:
            last_moment.append(int(df_match.iloc[j]['minute']))
        else:
            moments_now = sorted(moments_now)
            last_moment.append(int(df_match.iloc[j]['minute']) - moments_now[-1])

df_keys['time previous moment'] = last_moment

In [ ]:
#Add the dummy variables for the key moments
favorable = []
unfavorable = []
neutral = []
for i in range(len(df_keys)):
    if df_keys.iloc[i]['key_moment'] == 'favorable':
        favorable.append(1)
    else:
        favorable.append(0)
    if df_keys.iloc[i]['key_moment'] == 'unfavorable':
        unfavorable.append(1)
    else:
        unfavorable.append(0)   
    if df_keys.iloc[i]['key_moment'] == 'neutral':
        neutral.append(1)
    else:
        neutral.append(0) 
    

df_keys['favorable'] = favorable
df_keys['unfavorable'] = unfavorable
df_keys['neutral'] = neutral

In [ ]:
#select necessary columns
import pandas as pd
df_keys = df_keys.dropna()
df = pd.DataFrame(df_keys,columns=['team', 'type_key', 'mean pc home 2min before', 'mean pc away 2min before', 'mean pc home 2min after', 'mean pc away 2min after','delta pc momentum home', 'delta pc momentum away', 'delta pc home', 'delta pc away', 'var pc home', 'var pc away', 'index 2min before', 'index 2min after','home score', 'away score', 'score difference home', 'score difference away', 'time left','time previous moment', 'favorable', 'unfavorable', 'neutral'])


In [ ]:
#Run the regression for PC
from progressbar import ProgressBar
import statsmodels.api as sm
import random
beta = []
pbar = ProgressBar()
#iterate the regression a thousand times
for i in pbar(range(1000)):
    deltamom = []
    favorable = []
    unfavorable = []
    score_difference = []
    time_left = []
    deltapc = []
    varpc = []
    time_previous_moment = []
    #resample the types of key moments
    df_neutral = df.loc[df['neutral'] == 1]
    df_favorable = df.loc[df['favorable'] == 1]
    df_unfavorable = df.loc[df['unfavorable'] == 1]
    df_neutral = df_neutral.sample(frac=0.8, replace=True)
    df_favorable = df_favorable.sample(frac=0.8, replace=True)
    df_unfavorable = df_unfavorable.sample(frac=0.8, replace=True)
    df_new = pd.concat([df_neutral, df_favorable, df_unfavorable])
    #randomly take one of the teams' perspective
    for j in range(len(df_new)):
        number = random.randint(0,1)
        if number == 0:
            deltamom.append(df_new.iloc[j]['delta pc momentum home'])
            favorable.append(df_new.iloc[j]['favorable'])
            unfavorable.append(df_new.iloc[j]['unfavorable'])
            score_difference.append(df_new.iloc[j]['score difference home'])
            time_left.append(df_new.iloc[j]['time left'])
            deltapc.append(df_new.iloc[j]['delta pc home'])
            varpc.append(df_new.iloc[j]['var pc home'])
            time_previous_moment.append(df_new.iloc[j]['time previous moment'])
        else:
            deltamom.append(df_new.iloc[j]['delta pc momentum away'])
            favorable.append(df_new.iloc[j]['unfavorable'])
            unfavorable.append(df_new.iloc[j]['favorable'])
            score_difference.append(df_new.iloc[j]['score difference away'])
            time_left.append(df_new.iloc[j]['time left'])
            deltapc.append(df_new.iloc[j]['delta pc away'])
            varpc.append(df_new.iloc[j]['var pc away'])
            time_previous_moment.append(df_new.iloc[j]['time previous moment'])
    

    data = {'delta momentum': deltamom,
           'favorable': favorable,
           'unfavorable': unfavorable,
           'score difference': score_difference,
           'time left': time_left,
           'delta pc': deltapc,
           'variability pc': varpc,
           'time previous moment': time_previous_moment}
    df_reg = pd.DataFrame(data)
    
    #create the final dataframes for regression
    X = df_reg[['favorable', 'unfavorable', 'score difference', 'time left', 'delta pc', 'variability pc', 'time previous moment' ]]
    y = df_reg['delta momentum']

    #Run the regresison
    model = sm.OLS(y, X).fit()
    predictions = model.predict(X)

    #add the beta parameters to a list
    beta.append(model.params)

#create dataframe of the beta parameters
data = {'beta params':beta}
df_pc = pd.DataFrame(data)
     

In [ ]:
#ad the beta paramaters to a list
import statistics
favorable = []
unfavorable = []
score_difference = []
time_left = []
deltapc = []
varpc = []
shotorgoal = []
time_previous_moment = []
for i in range(len(df_pc)):
    favorable.append(df_pc.iloc[i]['beta params'][0])
    unfavorable.append(df_pc.iloc[i]['beta params'][1])
    score_difference.append(df_pc.iloc[i]['beta params'][2])
    time_left.append(df_pc.iloc[i]['beta params'][3])
    deltapc.append(df_pc.iloc[i]['beta params'][4])
    varpc.append(df_pc.iloc[i]['beta params'][5])
    time_previous_moment.append(df_pc.iloc[i]['beta params'][6])

In [ ]:
#round the beta coefficients so the mode can be found
favorable = [round(i,4) for i in favorable]
unfavorable = [round(i,4) for i in unfavorable]
score_difference = [round(i,4) for i in score_difference]
time_left = [round(i,4) for i in time_left]
deltapc = [round(i,4) for i in deltapc]
varpc = [round(i,4) for i in varpc]
time_previous_moment = [round(i,4) for i in time_previous_moment]

In [ ]:
#find the mode of the beta coefficients
from statistics import mean
beta_favorable = mean(statistics.multimode(favorable))
beta_unfavorable = mean(statistics.multimode(unfavorable))
beta_score_difference = mean(statistics.multimode(score_difference))
beta_time_left = mean(statistics.multimode(time_left))
beta_deltapc = mean(statistics.multimode(deltapc))
beta_varpc = mean(statistics.multimode(varpc))
beta_time_previous_moment = mean(statistics.multimode(time_previous_moment))

In [ ]:
#find the confidence intervals
confidence_intervals_favorable = [sorted(favorable)[25], sorted(favorable)[974]]
confidence_intervals_unfavorable = [sorted(unfavorable)[25], sorted(unfavorable)[974]]
confidence_intervals_score_difference = [sorted(score_difference)[25], sorted(score_difference)[974]]
confidence_intervals_time_left = [sorted(time_left)[25], sorted(time_left)[974]]
confidence_intervals_deltapc = [sorted(deltapc)[25], sorted(deltapc)[974]]
confidence_intervals_varpc = [sorted(varpc)[25], sorted(varpc)[974]]
confidence_intervals_time_previous_moment = [sorted(time_previous_moment)[25], sorted(time_previous_moment)[974]]

In [ ]:
#calculate the SE
SE_favorable = (confidence_intervals_favorable[1] - confidence_intervals_favorable[0])/(2*1.96)
SE_unfavorable = (confidence_intervals_unfavorable[1] - confidence_intervals_unfavorable[0])/(2*1.96)
SE_score_difference = (confidence_intervals_score_difference[1] - confidence_intervals_score_difference[0])/(2*1.96)
SE_time_left = (confidence_intervals_time_left[1] - confidence_intervals_time_left[0])/(2*1.96)
SE_deltapc = (confidence_intervals_deltapc[1] - confidence_intervals_deltapc[0])/(2*1.96)
SE_varpc = (confidence_intervals_varpc[1] - confidence_intervals_varpc[0])/(2*1.96)
SE_time_previous_moment = (confidence_intervals_time_previous_moment[1] - confidence_intervals_time_previous_moment[0])/(2*1.96)

In [ ]:
#calculate the t-test
#the p-values are retrieved from a table using this t-test
t_favorable = beta_favorable/SE_favorable
t_unfavorable = beta_unfavorable/SE_unfavorable
t_score_difference = beta_score_difference/SE_score_difference
t_time_left = beta_time_left/SE_time_left
t_deltapc = beta_deltapc/SE_deltapc
t_varpc = beta_varpc/SE_varpc
t_time_previous_moment = beta_time_previous_moment/SE_time_previous_moment

In [ ]:
#Calculate the MAE based on the beta coefficients
absolute_errors = []
for i in range(len(df)):
    predicted_y = beta_favorable * df.iloc[i]['favorable'] + beta_unfavorable * df.iloc[i]['unfavorable'] +  beta_score_difference * df.iloc[i]['score difference home'] + beta_time_left * df.iloc[i]['time left'] + beta_deltapc * df.iloc[i]['delta pc home'] + beta_varpc * df.iloc[i]['var pc home'] + beta_time_previous_moment * df.iloc[i]['time previous moment']
    real_y = df.iloc[i]['delta pc momentum home']
    absolute_errors.append(abs(predicted_y-real_y))
MAE = sum(absolute_errors)/len(df)

In [ ]:
#calculate the r-squared
from statistics import mean
sstot = []
ssres = []
mean_y = df['delta pc momentum home'].mean()
for i in range(len(df)):
    sstot.append((df.iloc[i]['delta pc momentum home'] - mean_y)**2)
    predicted_y = beta_favorable * df.iloc[i]['favorable'] + beta_unfavorable * df.iloc[i]['unfavorable'] + beta_score_difference * df.iloc[i]['score difference home'] + beta_time_left * df.iloc[i]['time left'] + beta_deltapc * df.iloc[i]['delta pc home'] + beta_varpc * df.iloc[i]['var pc home'] + beta_time_previous_moment * df.iloc[i]['time previous moment']
    ssres.append((df.iloc[i]['delta pc momentum home'] - predicted_y)**2)
    
rsquared = 1 - (sum(ssres)/sum(sstot))
adjrsquared = 1 - (((1-rsquared)*(len(df)-1))/(len(df)-10-1))
degrees_freedom_residuals = len(df) - 10 - 1
    

In [ ]:
#select necessary columns for PCV
df_keys = df_keys.dropna()
df = pd.DataFrame(df_keys,columns=['team', 'type_key', 'mean pcv home 2min before', 'mean pcv away 2min before', 'mean pcv home 2min after', 'mean pcv away 2min after','delta pcv momentum home', 'delta pcv momentum away', 'delta pcv home', 'delta pcv away', 'var pcv home', 'var pcv away', 'index 2min before', 'index 2min after','home score', 'away score', 'score difference home', 'score difference away', 'time left','time previous moment', 'favorable', 'unfavorable', 'neutral'])

In [ ]:
#Regression PCV
#these calculations are the same as in the PC model
beta = []

pbar = ProgressBar()
for i in pbar(range(1000)):
    deltamom = []
    favorable = []
    unfavorable = []
    score_difference = []
    time_left = []
    deltapcv = []
    varpcv = []
    time_previous_moment = []
    
    df_neutral = df.loc[df['neutral'] == 1]
    df_favorable = df.loc[df['favorable'] == 1]
    df_unfavorable = df.loc[df['unfavorable'] == 1]
    df_neutral = df_neutral.sample(frac=0.8, replace=True)
    df_favorable = df_favorable.sample(frac=0.8, replace=True)
    df_unfavorable = df_unfavorable.sample(frac=0.8, replace=True)
    df_new = pd.concat([df_neutral, df_favorable, df_unfavorable])
    for j in range(len(df_new)):
        number = random.randint(0,1)
        if number == 0:
            deltamom.append(df_new.iloc[j]['delta pcv momentum home'])
            favorable.append(df_new.iloc[j]['favorable'])
            unfavorable.append(df_new.iloc[j]['unfavorable'])
            score_difference.append(df_new.iloc[j]['score difference home'])
            time_left.append(df_new.iloc[j]['time left'])
            deltapcv.append(df_new.iloc[j]['delta pcv home'])
            varpcv.append(df_new.iloc[j]['var pcv home'])
            time_previous_moment.append(df_new.iloc[j]['time previous moment'])
        else:
            deltamom.append(df_new.iloc[j]['delta pcv momentum away'])
            favorable.append(df_new.iloc[j]['unfavorable'])
            unfavorable.append(df_new.iloc[j]['favorable'])
            score_difference.append(df_new.iloc[j]['score difference away'])
            time_left.append(df_new.iloc[j]['time left'])
            deltapcv.append(df_new.iloc[j]['delta pcv away'])
            varpcv.append(df_new.iloc[j]['var pcv away'])
            time_previous_moment.append(df_new.iloc[j]['time previous moment'])            
    

    data = {'delta momentum': deltamom,
           'favorable': favorable,
           'unfavorable': unfavorable,
           'score difference': score_difference,
           'time left': time_left,
           'delta pcv': deltapcv,
           'variability pcv': varpcv,
           'time previous moment': time_previous_moment}

    df_reg = pd.DataFrame(data)
    
    X = df_reg[['favorable', 'unfavorable', 'score difference', 'time left', 'delta pcv', 'variability pcv', 'time previous moment' ]]
    y = df_reg['delta momentum']

    # Note the difference in argument order
    model = sm.OLS(y, X).fit()
    predictions = model.predict(X) # make the predictions by the model

    # Print out the statistics
    beta.append(model.params)

data = {'beta params':beta}
df_pcv = pd.DataFrame(data)
     

In [ ]:
pickle.dump(df_pcv, open('regressionkeymomentspcv.p', 'wb')) 

In [ ]:
import statistics
favorable_pcv = []
unfavorable_pcv = []
score_difference_pcv = []
time_left_pcv = []
end_value = []
deltapcv = []
varpcv = []
shotorgoal_pcv = []
time_previous_moment_pcv = []
for i in range(len(df_pcv)):
    favorable_pcv.append(df_pcv.iloc[i]['beta params'][0])
    unfavorable_pcv.append(df_pcv.iloc[i]['beta params'][1])
    score_difference_pcv.append(df_pcv.iloc[i]['beta params'][2])
    time_left_pcv.append(df_pcv.iloc[i]['beta params'][3])
    deltapcv.append(df_pcv.iloc[i]['beta params'][4])
    varpcv.append(df_pcv.iloc[i]['beta params'][5])
    time_previous_moment_pcv.append(df_pcv.iloc[i]['beta params'][6])

In [ ]:
favorable_pcv = [round(i,4) for i in favorable_pcv]
unfavorable_pcv = [round(i,4) for i in unfavorable_pcv]
score_difference_pcv = [round(i,4) for i in score_difference_pcv]
time_left_pcv = [round(i,4) for i in time_left_pcv]
deltapcv = [round(i,4) for i in deltapcv]
varpcv = [round(i,4) for i in varpcv]
time_previous_moment_pcv = [round(i,4) for i in time_previous_moment_pcv]

In [ ]:
#betavariabelen
beta_favorable_pcv = mean(statistics.multimode(favorable_pcv))
beta_unfavorable_pcv = mean(statistics.multimode(unfavorable_pcv))
beta_score_difference_pcv = mean(statistics.multimode(score_difference_pcv))
beta_time_left_pcv = mean(statistics.multimode(time_left_pcv))
beta_deltapcv = mean(statistics.multimode(deltapcv))
beta_varpcv = mean(statistics.multimode(varpcv))
beta_time_previous_moment_pcv = mean(statistics.multimode(time_previous_moment_pcv))

In [ ]:
confidence_intervals_favorable_pcv = [sorted(favorable_pcv)[25], sorted(favorable_pcv)[974]]
confidence_intervals_unfavorable_pcv = [sorted(unfavorable_pcv)[25], sorted(unfavorable_pcv)[974]]
confidence_intervals_score_difference_pcv = [sorted(score_difference_pcv)[25], sorted(score_difference_pcv)[974]]
confidence_intervals_time_left_pcv = [sorted(time_left_pcv)[25], sorted(time_left_pcv)[974]]
confidence_intervals_deltapcv = [sorted(deltapcv)[25], sorted(deltapcv)[974]]
confidence_intervals_varpcv = [sorted(varpcv)[25], sorted(varpcv)[974]]
confidence_intervals_time_previous_moment_pcv = [sorted(time_previous_moment_pcv)[25], sorted(time_previous_moment_pcv)[974]]

In [ ]:
SE_favorable_pcv = (confidence_intervals_favorable_pcv[1] - confidence_intervals_favorable_pcv[0])/(2*1.96)
SE_unfavorable_pcv = (confidence_intervals_unfavorable_pcv[1] - confidence_intervals_unfavorable_pcv[0])/(2*1.96)
SE_score_difference_pcv = (confidence_intervals_score_difference_pcv[1] - confidence_intervals_score_difference_pcv[0])/(2*1.96)
SE_time_left_pcv = (confidence_intervals_time_left_pcv[1] - confidence_intervals_time_left_pcv[0])/(2*1.96)
SE_deltapcv = (confidence_intervals_deltapcv[1] - confidence_intervals_deltapcv[0])/(2*1.96)
SE_varpcv = (confidence_intervals_varpcv[1] - confidence_intervals_varpcv[0])/(2*1.96)
SE_time_previous_moment_pcv = (confidence_intervals_time_previous_moment_pcv[1] - confidence_intervals_time_previous_moment_pcv[0])/(2*1.96)

In [ ]:
t_favorable_pcv = beta_favorable_pcv/SE_favorable_pcv
t_unfavorable_pcv = beta_unfavorable_pcv/SE_unfavorable_pcv
t_score_difference_pcv = beta_score_difference_pcv/SE_score_difference_pcv
t_time_left_pcv = beta_time_left_pcv/SE_time_left_pcv
t_deltapcv = beta_deltapcv/SE_deltapcv
t_varpcv = beta_varpcv/SE_varpcv
t_time_previous_moment_pcv = beta_time_previous_moment_pcv/SE_time_previous_moment_pcv

In [ ]:
#MAE
absolute_errors = []
for i in range(len(df)):
    predicted_y = beta_favorable_pcv * df.iloc[i]['favorable'] + beta_unfavorable_pcv * df.iloc[i]['unfavorable'] + beta_score_difference_pcv * df.iloc[i]['score difference home'] + beta_time_left_pcv * df.iloc[i]['time left'] + beta_deltapcv * df.iloc[i]['delta pcv home'] + beta_varpcv * df.iloc[i]['var pcv home'] + beta_time_previous_moment_pcv * df.iloc[i]['time previous moment']
    real_y = df.iloc[i]['delta pcv momentum home']
    absolute_errors.append(abs(predicted_y-real_y))
MAE_pcv = sum(absolute_errors)/len(df)

In [ ]:
from statistics import mean
sstot = []
ssres = []
mean_y = df['delta pcv momentum home'].mean()
for i in range(len(df)):
    sstot.append((df.iloc[i]['delta pcv momentum home'] - mean_y)**2)
    predicted_y = beta_favorable_pcv * df.iloc[i]['favorable'] + beta_unfavorable_pcv * df.iloc[i]['unfavorable'] + beta_score_difference_pcv * df.iloc[i]['score difference home'] + beta_time_left_pcv * df.iloc[i]['time left'] + beta_deltapcv * df.iloc[i]['delta pcv home'] + beta_varpcv * df.iloc[i]['var pcv home'] + beta_time_previous_moment_pcv * df.iloc[i]['time previous moment']
    ssres.append((df.iloc[i]['delta pcv momentum home'] - predicted_y)**2)
    
rsquared_pcv = 1 - (sum(ssres)/sum(sstot))
adjrsquared_pcv = 1 - (((1-rsquared_pcv)*(len(df)-1))/(len(df)-10-1))
degrees_freedom_residuals_pcv = len(df) - 10 - 1
    

In [ ]:
#import the regression dataset for xT
infile = open('input xT regression key moments on Momentum.p','rb')
df_keys = pickle.load(infile)
infile.close()

In [ ]:
#add score difference
matches = df_keys.match_id.unique()
home_goals_complete = []
away_goals_complete = []
for i in range(len(matches)):
    df_shots = df_keys.loc[df_keys['match_id'] == matches[i]]    
    goals = []
    for j in range(len(df_shots)):
        if df_shots.iloc[j]['type_id'] == '16':
            goals.append([int(df_shots.iloc[j]['minute']) + int(df_shots.iloc[j]['seconds'])/60, df_shots.iloc[j]['team']])
    home_goals = []
    away_goals = []
    for j in range(len(df_shots)):
        home = 0
        away = 0
        for x in range(len(goals)):
            if (int(df_shots.iloc[j]['minute']) + int(df_shots.iloc[j]['seconds'])/60) >= goals[x][0] and goals[x][1] == 'Home':
                home = home + 1
            elif (int(df_shots.iloc[j]['minute']) + int(df_shots.iloc[j]['seconds'])/60) >= goals[x][0] and goals[x][1] == 'Away':
                away = away + 1
        home_goals.append(home)
        away_goals.append(away)
    home_goals_complete.extend(home_goals)
    away_goals_complete.extend(away_goals)

df_keys['home score'] = home_goals_complete
df_keys['away score'] = away_goals_complete
df_keys['score difference home'] = df_keys['home score'] - df_keys['away score']
df_keys['score difference away'] = df_keys['away score'] - df_keys['home score']

                
                

                
                

In [ ]:
#add time left
time_left = []
for i in range(len(df_keys)):
    time = int(df_keys.iloc[i]['minute']) + int(df_keys.iloc[i]['seconds'])/60
    time_left.append(90-time)
df_keys['time left'] = time_left

In [ ]:
#add time elapsed since previous shot or goal
matches = df_keys.match_id.unique()
last_moment = []
for i in range(len(matches)):
    df_match = df_keys.loc[df_keys['match_id'] == matches[i]]
    moments = []
    for j in range(len(df_match)):
        moments.append(int(df_match.iloc[j]['minute']))
    for j in range(len(df_match)):
        moments_now = [x for x in moments if x<int(df_match.iloc[j]['minute'])]
        if len(moments_now) == 0:
            last_moment.append(int(df_match.iloc[j]['minute']))
        else:
            moments_now = sorted(moments_now)
            last_moment.append(int(df_match.iloc[j]['minute']) - moments_now[-1])

df_keys['time previous moment'] = last_moment

In [ ]:
#create dummy variables
favorable = []
unfavorable = []
neutral = []
for i in range(len(df_keys)):
    if df_keys.iloc[i]['key_moment'] == 'favorable':
        favorable.append(1)
    else:
        favorable.append(0)
    if df_keys.iloc[i]['key_moment'] == 'unfavorable':
        unfavorable.append(1)
    else:
        unfavorable.append(0)   
    if df_keys.iloc[i]['key_moment'] == 'neutral':
        neutral.append(1)
    else:
        neutral.append(0) 
    

df_keys['favorable'] = favorable
df_keys['unfavorable'] = unfavorable
df_keys['neutral'] = neutral

In [ ]:
#select necessary columns
df_keys = df_keys.dropna()
df = pd.DataFrame(df_keys,columns=['team', 'type_key', 'mean xt home 2min before', 'mean xt away 2min before', 'mean xt home 2min after', 'mean xt away 2min after','delta mom home', 'delta mom away', 'delta xt home before', 'delta xt away before', 'var xt home before', 'var xt away before', 'index 2min before', 'index 2min after','home score', 'away score', 'score difference home', 'score difference away', 'time left','time previous moment', 'favorable', 'unfavorable', 'neutral'])

In [ ]:
#regression calculations xT
#same as pc and pcv
beta = []

pbar = ProgressBar()
for i in pbar(range(1000)):
    deltamom = []
    favorable = []
    unfavorable = []
    score_difference = []
    time_left = []
    deltaxt = []
    varxt = []
    time_previous_moment = []
    
    df_neutral = df.loc[df['neutral'] == 1]
    df_favorable = df.loc[df['favorable'] == 1]
    df_unfavorable = df.loc[df['unfavorable'] == 1]
    df_neutral = df_neutral.sample(frac=0.8, replace=True)
    df_favorable = df_favorable.sample(frac=0.8, replace=True)
    df_unfavorable = df_unfavorable.sample(frac=0.8, replace=True)
    df_new = pd.concat([df_neutral, df_favorable, df_unfavorable])
    for j in range(len(df_new)):
        number = random.randint(0,1)
        if number == 0:
            deltamom.append(df_new.iloc[j]['delta mom home'])
            favorable.append(df_new.iloc[j]['favorable'])
            unfavorable.append(df_new.iloc[j]['unfavorable'])
            score_difference.append(df_new.iloc[j]['score difference home'])
            time_left.append(df_new.iloc[j]['time left'])
            deltaxt.append(df_new.iloc[j]['delta xt home before'])
            varxt.append(df_new.iloc[j]['var xt home before'])
            time_previous_moment.append(df_new.iloc[j]['time previous moment'])
        else:
            deltamom.append(df_new.iloc[j]['delta mom away'])
            favorable.append(df_new.iloc[j]['unfavorable'])
            unfavorable.append(df_new.iloc[j]['favorable'])
            score_difference.append(df_new.iloc[j]['score difference away'])
            time_left.append(df_new.iloc[j]['time left'])
            deltaxt.append(df_new.iloc[j]['delta xt away before'])
            varxt.append(df_new.iloc[j]['var xt away before'])
            time_previous_moment.append(df_new.iloc[j]['time previous moment'])
            
    

    data = {'delta momentum': deltamom,
           'favorable': favorable,
           'unfavorable': unfavorable,
           'score difference': score_difference,
           'time left': time_left,
           'delta xt': deltaxt,
           'variability xt': varxt,
           'time previous moment': time_previous_moment}
    df_reg = pd.DataFrame(data)
    
    X = df_reg[['favorable', 'unfavorable', 'score difference', 'time left', 'delta xt', 'variability xt', 'time previous moment' ]]
    y = df_reg['delta momentum']

    model = sm.OLS(y, X).fit()
    predictions = model.predict(X)

    beta.append(model.params)

data = {'beta params':beta}
df_xt = pd.DataFrame(data)
     

In [ ]:
import statistics
favorable_xt = []
unfavorable_xt = []
score_difference_xt = []
time_left_xt = []
end_value = []
deltaxt = []
varxt = []
shotorgoal_xt = []
time_previous_moment_xt = []
for i in range(len(df_xt)):
    favorable_xt.append(df_xt.iloc[i]['beta params'][0])
    unfavorable_xt.append(df_xt.iloc[i]['beta params'][1])
    score_difference_xt.append(df_xt.iloc[i]['beta params'][2])
    time_left_xt.append(df_xt.iloc[i]['beta params'][3])
    deltaxt.append(df_xt.iloc[i]['beta params'][4])
    varxt.append(df_xt.iloc[i]['beta params'][5])
    time_previous_moment_xt.append(df_xt.iloc[i]['beta params'][6])

In [ ]:
favorable_xt = [round(i,4) for i in favorable_xt]
unfavorable_xt = [round(i,4) for i in unfavorable_xt]
score_difference_xt = [round(i,4) for i in score_difference_xt]
time_left_xt = [round(i,4) for i in time_left_xt]
deltaxt = [round(i,4) for i in deltaxt]
varxt = [round(i,4) for i in varxt]
time_previous_moment_xt = [round(i,4) for i in time_previous_moment_xt]

In [ ]:
#betavariabelen
beta_favorable_xt = mean(statistics.multimode(favorable_xt))
beta_unfavorable_xt = mean(statistics.multimode(unfavorable_xt))
beta_score_difference_xt = mean(statistics.multimode(score_difference_xt))
beta_time_left_xt = mean(statistics.multimode(time_left_xt))
beta_deltaxt = mean(statistics.multimode(deltaxt))
beta_varxt = mean(statistics.multimode(varxt))
beta_time_previous_moment_xt = mean(statistics.multimode(time_previous_moment_xt))

In [ ]:
confidence_intervals_favorable_xt = [sorted(favorable_xt)[25], sorted(favorable_xt)[974]]
confidence_intervals_unfavorable_xt = [sorted(unfavorable_xt)[25], sorted(unfavorable_xt)[974]]
confidence_intervals_score_difference_xt = [sorted(score_difference_xt)[25], sorted(score_difference_xt)[974]]
confidence_intervals_time_left_xt = [sorted(time_left_xt)[25], sorted(time_left_xt)[974]]
confidence_intervals_deltaxt = [sorted(deltaxt)[25], sorted(deltaxt)[974]]
confidence_intervals_varxt = [sorted(varxt)[25], sorted(varxt)[974]]
confidence_intervals_time_previous_moment_xt = [sorted(time_previous_moment_xt)[25], sorted(time_previous_moment_xt)[974]]

In [ ]:
SE_favorable_xt = (confidence_intervals_favorable_xt[1] - confidence_intervals_favorable_xt[0])/(2*1.96)
SE_unfavorable_xt = (confidence_intervals_unfavorable_xt[1] - confidence_intervals_unfavorable_xt[0])/(2*1.96)
SE_score_difference_xt = (confidence_intervals_score_difference_xt[1] - confidence_intervals_score_difference_xt[0])/(2*1.96)
SE_time_left_xt = (confidence_intervals_time_left_xt[1] - confidence_intervals_time_left_xt[0])/(2*1.96)
SE_deltaxt = (confidence_intervals_deltaxt[1] - confidence_intervals_deltaxt[0])/(2*1.96)
SE_varxt = (confidence_intervals_varxt[1] - confidence_intervals_varxt[0])/(2*1.96)
SE_time_previous_moment_xt = (confidence_intervals_time_previous_moment_xt[1] - confidence_intervals_time_previous_moment_xt[0])/(2*1.96)

In [ ]:
t_favorable_xt = beta_favorable_xt/SE_favorable_xt
t_unfavorable_xt = beta_unfavorable_xt/SE_unfavorable_xt
t_score_difference_xt = beta_score_difference_xt/SE_score_difference_xt
t_time_left_xt = beta_time_left_xt/SE_time_left_xt
t_deltaxt = beta_deltaxt/SE_deltaxt
t_varxt = beta_varxt/SE_varxt
t_time_previous_moment_xt = beta_time_previous_moment_xt/SE_time_previous_moment_xt

In [ ]:
#MAE
absolute_errors = []
for i in range(len(df)):
    predicted_y = beta_favorable_xt * df.iloc[i]['favorable'] + beta_unfavorable_xt * df.iloc[i]['unfavorable'] + beta_score_difference_xt * df.iloc[i]['score difference home'] + beta_time_left_xt * df.iloc[i]['time left'] + beta_deltaxt * df.iloc[i]['delta xt home before'] + beta_varxt * df.iloc[i]['var xt home before'] + beta_time_previous_moment_xt * df.iloc[i]['time previous moment']
    real_y = df.iloc[i]['delta mom home']
    absolute_errors.append(abs(predicted_y-real_y))
MAE_xt = sum(absolute_errors)/len(df)

In [ ]:
from statistics import mean
sstot = []
ssres = []
mean_y = df['delta mom home'].mean()
for i in range(len(df)):
    sstot.append((df.iloc[i]['delta mom home'] - mean_y)**2)
    predicted_y = beta_favorable_xt * df.iloc[i]['favorable'] + beta_unfavorable_xt * df.iloc[i]['unfavorable'] + beta_score_difference_xt * df.iloc[i]['score difference home'] + beta_time_left_xt * df.iloc[i]['time left'] + beta_deltaxt * df.iloc[i]['delta xt home before'] + beta_varxt * df.iloc[i]['var xt home before'] + beta_time_previous_moment_xt * df.iloc[i]['time previous moment']
    ssres.append((df.iloc[i]['delta mom home'] - predicted_y)**2)
    
rsquared_xt = 1 - (sum(ssres)/sum(sstot))
adjrsquared_xt = 1 - (((1-rsquared_xt)*(len(df)-1))/(len(df)-10-1))
degrees_freedom_residuals_xt = len(df) - 10 - 1
    